In [553]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil

In [554]:
today = date.today()

In [555]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [556]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento [0-9]+\/[0-9]+', 'Directiva [0-9]+\/[0-9]+','Directiva \(UE\) [0-9]+\/[0-9]+','Reglamento n\º [0-9]+\/[0-9]+', 'Reglamento \(UE\) [0-9]+\/[0-9]+','Reglamento \(UE\) n\º [0-9]+\/[0-9]+','Reglamento \(CE\) n\º [0-9]+\/[0-9]+','Reglamento \(CEE\) n\º [0-9]+\/[0-9]+','Reglamento de Ejecución \(UE\) [0-9]+\/[0-9]+','Reglamento de Ejecución \(UE\) n\º [0-9]+\/[0-9]+' ,'Reglamento de Ejecución [0-9]+\/[0-9]+','Reglamento Delegado [0-9]+\/[0-9]+','Reglamento Delegado \(UE\) [0-9]+\/[0-9]+','Reglamento Delegado \(UE\) n\º [0-9]+\/[0-9]+','Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión [0-9]+\/[0-9]+','Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']

## Importa BBDD ASECORP

In [557]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [558]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [562]:
#ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'Europa']

In [565]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'].isin (['España','Europa'])]

In [566]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18496,STC 86/2019 ESP,86,"Sentencia 86/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18497,STC 87/2019 ESP,87,"Sentencia 87/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18498,STC 88/2018 ESP,88,"Sentencia 88/2018, de 19 de julio de 2018. Con...",ESP,España,NaN,M. Ambiente / Reg. Industrial,[1][3]
18499,STC 9/3/2010 ESP,9/3/2010,"Sentencia de 9 de marzo de 2010, de la Sala Te...",ESP,España,NaN,M. Ambiente,[1]


In [585]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [568]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [569]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
4699,DIR 2013/51 EUR,51,"Directiva 2013/51/EURATOM del Consejo, de 22 d...",EUR,Europa,NaN,M. Ambiente / Prevención,[1][2],[Directiva 2013/51]
4700,DIR 2013/52 EUR,52,"Directiva 2013/52/UE de la Comisión, de 30 de ...",EUR,Europa,NaN,M. Ambiente / Prevención,[1][2],"[Directiva 2013/52, Directiva 96/98]"
4701,DIR 2013/56 EUR,56,Directiva 2013/56/UE del Parlamento Europeo y ...,EUR,Europa,NaN,M. Ambiente,[1],"[Directiva 2013/56, Directiva 2006/66, Decisió..."
4702,DIR 2013/59 EUR,59,"Directiva 2013/59/Euratom del Consejo, de 5 de...",EUR,Europa,NaN,Prevención,[2],[Directiva 2013/59]
4703,DIR 2013/64 EUR,64,Directiva 2013/64/UE del consejo de 17 de dici...,EUR,Europa,NaN,M. Ambiente / Prevención,[1][2],[Directiva 2013/64]
4704,DIR 2014/100 EUR,100,Directiva 2014/100/UE de la Comisión de 28 de ...,EUR,Europa,NaN,M. Ambiente / Prevención,[1][2],"[Directiva 2014/100, Directiva 2002/59]"
4705,DIR 2014/101 EUR,101,Directiva 2014/101/UE de la Comisión de 30 de ...,EUR,Europa,NaN,M. Ambiente,[1],"[Directiva 2014/101, Directiva 2000/60]"
4706,DIR 2014/102 EUR,102,Directiva 2014/102/UE del consejo de 7 de novi...,EUR,Europa,NaN,M. Ambiente,[1],"[Directiva 2014/102, Directiva 2013/34]"
4707,DIR 2014/27 EUR,27,Directiva 2014/27/UE del Parlamento Europeo Y ...,EUR,Europa,NaN,Prevención,[2],"[Directiva 2014/27, Directiva 2004/37, Reglame..."
4708,DIR 2014/29 EUR,29,Directiva 2014/29/UE del parlamento europeo y ...,EUR,Europa,NaN,Reg. Industrial,[3],[Directiva 2014/29]


In [570]:
# titulo

In [571]:
len(titulo)

4849

In [572]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [573]:
boletin[0:25]

["['Acuerdo Multilateral RID 1/2020']",
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 2/2020']",
 '[]',
 "['Acuerdo Multilateral RID 1/2013']",
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 3/2007']",
 "['Acuerdo Multilateral RID 4/2009']",
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [574]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [575]:
n_vacios

594

In [576]:
vacios[0:25]

['Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre el traslado de residuos, hecho en Madrid el 29 de noviembre de 2011.',
 'Canje de Notas por el que se procede a la prórroga del mantenimiento en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 27 de enero de 2000.',
 'Aplicación provisional del

In [577]:
boletin[0:25]

[['Acuerdo Multilateral RID 1/2020'],
 ['Acuerdo Multilateral RID 2/2020'],
 ['Acuerdo Multilateral RID 1/2013'],
 ['Acuerdo Multilateral RID 3/2007'],
 ['Acuerdo Multilateral RID 4/2009'],
 ['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268']]

In [578]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [579]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral RID 1/2020',
 'Acuerdo Multilateral RID 2/2020',
 'Acuerdo Multilateral RID 1/2013',
 'Acuerdo Multilateral RID 3/2007',
 'Acuerdo Multilateral RID 4/2009',
 'Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268']

In [580]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [581]:
boletin_ASECORP_flat_list[0:25]

['Sentencia 15/2018',
 'Resolución de 27 de abril de 2005',
 'Real Decreto 201/2002',
 'Real Decreto Legislativo 1/2007',
 'Orden AAA/2272/2013',
 'Orden ITC/633/2007',
 'Reglamento 1488/94',
 'Real Decreto 2001/1995',
 'Resolución de 16 de marzo de 2011',
 'Real Decreto 180/2008',
 'Reglamento (UE) nº 984/2013',
 'Ley 28/2015',
 'Orden PRE/3290/2002',
 'Orden ITC/3520/2009',
 'Reglamento (UE) 2020/741',
 'Reglamento (UE) 2017/752',
 'Real Decreto 2351/2004',
 'Sentencia de 24 de enero de 2011',
 'Reglamento (CE) nº 641/2009',
 'Real Decreto 679/2006',
 'Directiva 2002/49',
 'Resolución de 24 de julio de 1989',
 'Real Decreto 1720/2007',
 'Reglamento (CE) nº 1907/2006',
 'Resolución de 18 de marzo de 2020']

# Genera Fichero EXCEL de resultados

In [582]:
## Crea función que convierte lista a string en todas las columnas de tabla_analisis
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [583]:
## Aplica función de conversión de listas a strings
ASECORP_BBDD_BOE_final = ASECORP_BBDD_BOE.apply(lambda x: [list2Str(i) for i in x])

In [584]:
ASECORP_BBDD_BOE_final.to_csv("./ASECORP/ASECORP_BBDD_Tageada_" + today.strftime("%Y%m%d") + ".csv", index=False) 